In [63]:
import json
import pandas as pd
import os
from tqdm import tqdm
from dotenv import load_dotenv

load_dotenv()
GEMINI_KEY = os.getenv("GEMINI_API_KEY")
MISTRAL_KEY = os.getenv("MISTRAL_API_KEY")
input_file = "body_image.json"

with open(input_file, "r", encoding="utf-8") as f:
    prompts = json.load(f)

print("Loaded", len(prompts), "prompts")



Loaded 5 prompts


In [65]:
import google.generativeai as genai

In [67]:
genai.configure(api_key=GEMINI_KEY)
def call_gemini(prompt):
    try:
        model = genai.GenerativeModel("gemini-2.5-flash")
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"ERROR: {str(e)}"

In [69]:
from mistralai import Mistral

mistral_client = Mistral(api_key=MISTRAL_KEY)

def call_mistral(prompt):
    try:
        res = mistral_client.chat.complete(
            model="mistral-small-latest",
            messages=[{"role": "user", "content": prompt}],
            stream=False,
        )
        return res.choices[0].message.content
    except Exception as e:
        return f"ERROR (Mistral): {str(e)}"

In [ ]:
os.makedirs("outputs", exist_ok=True)
output_path = "outputs/body_image_outputs.jsonl"

with open(output_path, "w", encoding="utf-8") as fout:

    for p in tqdm(prompts):
        prompt_id = p["id"]
        prompt_text = p["prompt_text"]

        gem_output = call_gemini(prompt_text)

        mis_output = call_mistral(prompt_text)

        record = {
            "prompt_id": prompt_id,
            "category": p["category"],
            "prompt_text": prompt_text,
            "source_url": p["source_url"],
            "gemini_output": gem_output,
            "mistral_output": mis_output
        }

        fout.write(json.dumps(record, ensure_ascii=False) + "\n")

print("Done! JSONL saved to:", output_path)